In [8]:
from nestmodel.ergm import Gnp_row_first
from numba import njit
import numpy as np

In [27]:
@njit(cache=True)
def random_matrix(n_rows, n_columns, p):
    approx = int(n_rows*n_columns*p)
    E=np.empty((approx, 2), dtype=np.uint32)
    i=0
    
    x = -1;
    y = 0;
    k = 1;
    agg = 0;
    upper_bound = n_rows*n_columns;
    while True:
        k = np.random.geometric(p)
        x += k;
        agg += k;
        if agg > upper_bound:
            break;
        while x >= n_columns:
            x-=n_columns;
            y+=1;
            
        E[i,0]=y
        E[i,1]=x

        i+=1
        if i >= len(E):
            E2 = np.empty((len(E)+approx,2), dtype=np.uint32)
            E2[:len(E)] = E[:]
            E = E2

    return E[:i,:]

In [28]:
random_matrix(10,10, 0.1)

array([[0, 4],
       [1, 1],
       [3, 1],
       [4, 4],
       [5, 0],
       [5, 2],
       [5, 8],
       [6, 1],
       [7, 4],
       [7, 6],
       [9, 3]], dtype=uint32)

In [ ]:
def SBM(Omega, p):
    assert np.all(Omega==Omega.T)
    all_edges = []
    
    k = len(Omega)
    for i in range(k):
        for j in range(i, k):
            p = Omega[i,j]
            if i == j:
                new_edges = random_
            else:
            all_edges.append(new_edges)

In [42]:
def SBM(partition_sizes, P):
    
    def offset_edges(a,b, edge):
        edges[:,0]+=a
        edges[:,1]+=b
        return edges
        
    all_edges = []
    n_partitions = len(partition_sizes)
    ns_cumsum = np.array([0, *np.cumsum(partition_sizes)], dtype=np.int64)
    
    for i in range(n_partitions):
        for j in range(i+1):

            p= P[i,j]
            
            #print(p)
            if i==j:
                edges = Gnp_row_first(partition_sizes[j], p)
            else:
                edges = random_matrix(partition_sizes[j], partition_sizes[i], p)
            #print(edges)
            di = ns_cumsum[i]
            dj = ns_cumsum[j]

            offset_edges(dj,di, edges)
            all_edges.append(edges)
    
    return np.vstack(all_edges)

In [51]:
a = 0.4
b = 0.1
c = a
n=100
edges = SBM([n//2,n//2], np.array([[a,b],[b, c]]))

In [52]:
edges.shape

(1201, 2)

In [53]:
from nestmodel.fast_graph import FastGraph

In [93]:
from scipy.sparse import linalg
from tqdm.notebook import tqdm

In [97]:

out = []
for sample_id in tqdm(range(100)):
    for a in np.linspace(0.12,0.5,10):
        for n in [100,1000,10_000]:
            n=int(n)
            solution = np.hstack([-np.ones(n//2), np.ones(n//2)])
            c = a
            edges = SBM([n//2,n//2], np.array([[a,b],[b, c]]))
            A = FastGraph(edges, is_directed=False).to_coo()

            vals, vecs = linalg.eigsh(A, k=2, v0=np.ones(n))
            second_val = np.argmin(vals)
            pred = np.sign(vecs[:,second_val]).astype(int)
            correct = max(np.all(pred==solution), np.all(pred==-solution))
            out.append((sample_id, a, n, correct))

  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 